In [8]:
import os
import pandas as pd
import json

In [9]:
%run ../utils/__init__.py
%run ../metrics/__init__.py

In [10]:
classification = False

In [11]:
base_folder = os.path.join(WORKSPACE_DIR,
                           'classification' if classification else 'report_generation')
results_folder = os.path.join(base_folder, 'results')
# runs_folder = os.path.join(base_folder, 'runs')
# run_names = os.listdir(results_folder)
# _tb_run_names = os.listdir(runs_folder)
# run_names = [r for r in run_names if r in _tb_run_names or 'dummy' in r]
# run_names = list(run_names)
# if 'debug' in run_names: run_names.remove('debug')
# run_names

In [33]:
def load_results():
    all_results = []

    for run_name in os.listdir(results_folder): # run_names:
        if run_name == 'debug':
            continue
            
        if not run_name.startswith('0915'):
            continue

        folder = os.path.join(results_folder, run_name)
        for filename in os.listdir(folder):
            if not filename.endswith('json'):
                continue
            if filename.startswith('chexpert'):
                # For now, using only metrics.json
                continue
            
            filepath = os.path.join(folder, filename)

            if not os.path.isfile(filepath):
                continue

            with open(filepath, 'r') as f:
                results = json.load(f)

            additional_name = filename
            additional_name = additional_name.replace('chexpert-', '')
            additional_name = additional_name.replace('metrics', '')
            additional_name = additional_name.replace('.json', '')
            all_results.append((run_name, additional_name, results))

    return all_results

In [34]:
all_results = load_results()
len(all_results)

20

In [35]:
def create_results_df(results, key):
    run_names, results_dicts = zip(*[
        (f'{run}--{name}', d[key])
        for run, name, d in results if key in d
    ])
    
    df = pd.DataFrame(results_dicts, index=run_names)
    return df

In [36]:
# results_df_all = create_results_df(all_results, 'all')
results_df_train = create_results_df(all_results, 'train')
results_df_val = create_results_df(all_results, 'val')
results_df_test = create_results_df(all_results, 'test')

### Report generation

In [37]:
METRICS = ['bleu1', 'bleu2', 'bleu3', 'bleu4', 'bleu', 'rougeL', 'ciderD']

In [38]:
contains = 'dummy'
contains = '-free'

results_df_val = results_df_val.loc[results_df_val.index.str.contains(contains)]
results_df_test = results_df_test.loc[results_df_test.index.str.contains(contains)]

In [39]:
results_df_val

,loss,word_loss,stop_loss,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD,distinct_words,distinct_sentences,word_acc
0915_174026_dummy-common-sentences-100---free,-1.0,-1.0,-1.0,0.330119,0.201151,0.133937,9.366623e-02,0.189718,0.272445,0.160992,109,277,NaN
0915_172915_dummy-common-words-10---free,-1.0,NaN,NaN,0.313824,0.103350,0.020775,7.707264e-07,0.109487,0.238744,0.055413,10,2261,NaN
0915_173524_dummy-common-sentences-10---free,-1.0,-1.0,-1.0,0.263360,0.163589,0.109638,7.783674e-02,0.153606,0.288849,0.146738,18,91,NaN
0915_172709_dummy-constant---free,-1.0,NaN,NaN,0.436451,0.300677,0.218200,1.630881e-01,0.279604,0.349055,0.285976,22,7,NaN
0915_173609_dummy-random---free,-1.0,NaN,NaN,0.357511,0.200531,0.125502,8.512275e-02,0.192167,0.270692,0.205858,1003,1623,NaN
0915_173754_dummy-common-sentences-50---free,-1.0,-1.0,-1.0,0.317061,0.192819,0.127711,8.886443e-02,0.181614,0.274037,0.145440,79,162,NaN
0915_173307_dummy-common-words-100---free,-1.0,NaN,NaN,0.323063,0.077896,0.015750,4.184483e-03,0.105223,0.208364,0.110097,100,2911,NaN
0915_173110_dummy-common-words-50---free,-1.0,NaN,NaN,0.382632,0.105636,0.019846,7.378200e-07,0.127029,0.226366,0.081270,50,3521,NaN
0915_173951_dummy-most-similar-image_densenet-121---free,-1.0,NaN,NaN,0.363605,0.206306,0.130994,8.942934e-02,0.197584,0.279709,0.211664,942,1607,NaN
0915_174222_dummy-most-similar-image_0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid---free,-1.0,NaN,NaN,0.369602,0.217082,0.141778,9.985359e-02,0.207079,0.287151,0.221532,852,1368,NaN


In [40]:
results_df_test[METRICS]

,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0915_174026_dummy-common-sentences-100---free,0.346689,0.210850,0.138440,9.576881e-02,0.197937,0.280916,0.165551
0915_172915_dummy-common-words-10---free,0.312055,0.107749,0.024166,8.550974e-07,0.110993,0.243604,0.042768
0915_173524_dummy-common-sentences-10---free,0.275183,0.170479,0.112833,7.838136e-02,0.159219,0.294405,0.151610
0915_172709_dummy-constant---free,0.454623,0.311525,0.223393,1.653650e-01,0.288726,0.356681,0.292646
0915_173609_dummy-random---free,0.362480,0.196943,0.117449,7.459662e-02,0.187867,0.263606,0.111804
0915_173754_dummy-common-sentences-50---free,0.332812,0.205903,0.137483,9.688422e-02,0.193271,0.282765,0.188344
0915_173307_dummy-common-words-100---free,0.348717,0.089660,0.016369,6.481232e-07,0.113687,0.206325,0.071575
0915_173110_dummy-common-words-50---free,0.375112,0.102008,0.019032,7.256907e-07,0.124038,0.224006,0.075382
0915_173951_dummy-most-similar-image_densenet-121---free,0.381404,0.220458,0.142491,9.931259e-02,0.210916,0.284843,0.207753
0915_174222_dummy-most-similar-image_0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid---free,0.382749,0.220256,0.142339,9.958416e-02,0.211232,0.288225,0.230329


In [41]:
# print_columns = [
#     c
#     for c in results_df_test.columns
#     if c != 'loss' and 'cm' not in c and c != 'word_acc'
# ]
print((results_df_test).to_latex(columns=METRICS,
                                 float_format='%.3f',
                                ))

\begin{tabular}{lrrrrrrr}
\toprule
{} &  bleu1 &  bleu2 &  bleu3 &  bleu4 &  bleu &  rougeL &  ciderD \\
\midrule
0915\_174026\_dummy-common-sentences-100---free      &  0.347 &  0.211 &  0.138 &  0.096 & 0.198 &   0.281 &   0.166 \\
0915\_172915\_dummy-common-words-10---free           &  0.312 &  0.108 &  0.024 &  0.000 & 0.111 &   0.244 &   0.043 \\
0915\_173524\_dummy-common-sentences-10---free       &  0.275 &  0.170 &  0.113 &  0.078 & 0.159 &   0.294 &   0.152 \\
0915\_172709\_dummy-constant---free                  &  0.455 &  0.312 &  0.223 &  0.165 & 0.289 &   0.357 &   0.293 \\
0915\_173609\_dummy-random---free                    &  0.362 &  0.197 &  0.117 &  0.075 & 0.188 &   0.264 &   0.112 \\
0915\_173754\_dummy-common-sentences-50---free       &  0.333 &  0.206 &  0.137 &  0.097 & 0.193 &   0.283 &   0.188 \\
0915\_173307\_dummy-common-words-100---free          &  0.349 &  0.090 &  0.016 &  0.000 & 0.114 &   0.206 &   0.072 \\
0915\_173110\_dummy-common-words-50---free    

In [74]:
np.mean([0.369, 0.246, 0.171, 0.115])

0.22525

### Classification

In [64]:
# contains = 'covid-x'
# contains = 'cxr14'
contains = 'e0'
# contains = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid'
# contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
contains = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid' # WINNER

contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
contains = 'covid-uc'

In [71]:
results_df_train = results_df_train.loc[results_df_train.index.str.contains(contains)]
results_df_val = results_df_val.loc[results_df_val.index.str.contains(contains)]
results_df_test = results_df_test.loc[results_df_test.index.str.contains(contains)]

In [62]:
results_df_all

,loss,acc,prec_covid,prec_Non-COVID,prec_normal,recall_covid,recall_Non-COVID,recall_normal,spec_covid,spec_Non-COVID,spec_normal,cm
0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid---covid-uc_size256_frontal,3.348938,0.393496,0.050633,0.200000,0.941964,0.666667,0.202703,0.408124,0.492386,0.889094,0.867347,"[[15, 8, 1], [44, 19, 11], [229, 83, 205]]"
0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid---covid-uc_size512_frontal,4.755333,0.269919,0.046083,0.282051,0.950704,0.833333,0.148649,0.261122,0.299492,0.948244,0.928571,"[[20, 4, 0], [55, 12, 7], [342, 35, 140]]"
0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid---covid-uc_size512,2.975181,0.234428,0.038241,0.177340,0.961783,0.714286,0.367347,0.199472,0.411696,0.787261,0.952381,"[[18, 10, 0], [49, 42, 7], [406, 185, 166]]"
0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid---covid-uc_size256,2.332392,0.322763,0.033040,0.164773,0.952569,0.535714,0.295918,0.318362,0.486550,0.812739,0.904762,"[[11, 17, 0], [52, 35, 11], [332, 189, 236]]"


In [111]:
results_df_train.sort_values(['acc', 'recall_covid', 'prec_covid'],
                           ascending=False).dropna(axis=1, how='all')

KeyError: 'recall_covid'

In [72]:
results_df_val.sort_values(['acc', 'recall_covid', 'prec_covid'],
                           ascending=False).dropna(axis=1, how='all')

,loss,acc,prec_covid,prec_normal,recall_covid,recall_normal,spec_covid,spec_normal,cm,prec_Non-COVID,recall_Non-COVID,spec_Non-COVID
0722_193534_covid-uc_densenet-121_lr0.0001_os-max10_aug_size256--,0.421872,0.897727,0.000000,0.916667,0.0,0.974684,0.987952,0.222222,"[[0, 0, 5], [0, 2, 2], [1, 1, 77]]",0.666667,0.5,0.988095
0722_201835_covid-uc_densenet-121_lr0.0001_os-max10_aug_pre-covid-x--,1.011396,0.772727,0.181818,0.969697,0.8,0.810127,0.783133,0.777778,"[[4, 0, 1], [2, 0, 2], [13, 3, 63]]",0.000000,0.0,1.000000


In [67]:
results_df_test

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,...,cm_Infiltration,cm_Mass,cm_Nodule,cm_Pneumothorax,cm_Consolidation,cm_Edema,cm_Emphysema,cm_Fibrosis,cm_Pleural_Thickening,cm_Hernia


In [66]:
results_df_test.sort_values(['acc', 'recall_covid', 'prec_covid'],
                            ascending=False).dropna(axis=1, how='all')

""


In [36]:
print_columns = [
    'acc',
    'prec_covid','prec_pneumonia','prec_normal',
    'recall_covid','recall_pneumonia', 'recall_normal',
    'spec_covid','spec_pneumonia', 'spec_normal'
#     'acc_Pneumonia', 'prec_Pneumonia', 'recall_Pneumonia'
]
# print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]

In [37]:
print((results_df_test*100).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &   acc &  prec\_covid &  prec\_pneumonia &  prec\_normal &  recall\_covid &  recall\_pneumonia &  recall\_normal &  spec\_covid &  spec\_pneumonia &  spec\_normal \\
\midrule
0717\_101812\_covid-x\_densenet-121\_lr1e-06\_os-max... & 93.73 &       93.94 &           91.12 &        95.53 &         93.00 &             93.27 &          94.12 &       99.59 &           94.52 &        94.38 \\
\bottomrule
\end{tabular}



### Report-generation: results at different report lengths

In [32]:
vals_words = [20, 25, 27, 33, 44, None]
vals_sents = [3, 4, 5, 6, None]

In [40]:
max_words = vals_words[0]
suffix = f'max-words-{max_words}' if max_words else ''
all_results = load_results(suffix)
results_df_test = create_results_df(all_results, 'test')
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0717_041434_lstm_lr0.0001_densenet-121,5.579509,0.149129,0.478934,0.325275,0.256634,0.218300,0.319786,0.395486,1.354237
0716_211601_lstm-att_lr0.0001_densenet-121,7.382415,0.068293,0.442359,0.282490,0.199632,0.150046,0.268632,0.358223,0.775244
0717_015057_h-lstm_lr0.0001_densenet-121,4.548621,0.162518,0.328184,0.195510,0.126410,0.082401,0.183126,0.334782,0.256378
0716_234501_h-lstm-att_lr0.0001_densenet-121,5.157984,0.132504,0.298223,0.179060,0.115951,0.075574,0.167202,0.315891,0.101786
